# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Bayesian-Line-Identification-framework-for-WISPS-data" data-toc-modified-id="Bayesian-Line-Identification-framework-for-WISPS-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Bayesian Line Identification framework for WISPS data</a></div><div class="lev2 toc-item"><a href="#Retrieve-required-data-from-aXe-products" data-toc-modified-id="Retrieve-required-data-from-aXe-products-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Retrieve required data from aXe products</a></div>

# Bayesian Line Identification framework for WISPS data
## Retrieve required data from aXe products
The line identification framework requires:
* A postage stamp cutout from the direct image to be used as a two-dimensional template for the line flux.
* The drizzled, wavelength calibrated, contamination-subtracted stamp for the target.